In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils import data
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
print(device)

cuda


The AlexNet architecture consists of eight layers in total.

The first five layers are convolutional layers.
The sizes of the convolutional filters are 11×11, 5×5, 3×3, 3×3, and 3×3 for the respective convolutional layers.

Some of the convolutional layers are followed by max-pooling layers, which help reduce spatial dimensions while retaining important features.

The activation function used in the network is the Rectified Linear Unit (ReLU), known for its superior performance compared to sigmoid and tanh functions.

After the convolutional layers, there are three fully connected layers.
The network’s parameters can be tuned based on the training performance.


In [3]:
import numpy as np
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision import datasets, transforms

def get_train_valid_loader(data_dir,
                           batch_size,
                           augment,
                           random_seed,
                           valid_size=0.1,
                           shuffle=True):
    """
    Utility function to get training and validation data loaders.
    """
    normalize = transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010],
    )

    # Define transforms
    valid_transform = transforms.Compose([
        transforms.Resize((227, 227)),
        transforms.ToTensor(),
        normalize,
    ])

    if augment:
        train_transform = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.Resize((227, 227)),
            transforms.ToTensor(),
            normalize,
        ])
    else:
        train_transform = transforms.Compose([
            transforms.Resize((227, 227)),
            transforms.ToTensor(),
            normalize,
        ])

    # Load the dataset
    train_dataset = datasets.CIFAR10(
        root=data_dir, train=True,
        download=True, transform=train_transform,
    )

    valid_dataset = datasets.CIFAR10(
        root=data_dir, train=True,
        download=True, transform=valid_transform,
    )

    # Split the dataset into training and validation
    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    # Create data loaders
    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, sampler=train_sampler
    )

    valid_loader = DataLoader(
        valid_dataset, batch_size=batch_size, sampler=valid_sampler
    )

    return train_loader, valid_loader


def get_test_loader(data_dir, batch_size, shuffle=True):
    """
    Utility function to get test data loader.
    """
    normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    )

    # Define transform
    transform = transforms.Compose([
        transforms.Resize((227, 227)),
        transforms.ToTensor(),
        normalize,
    ])

    dataset = datasets.CIFAR10(
        root=data_dir, train=False,
        download=True, transform=transform,
    )

    data_loader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle
    )

    return data_loader


# CIFAR10 dataset
data_dir = './data'

train_loader, valid_loader = get_train_valid_loader(
    data_dir=data_dir,
    batch_size=64,
    augment=False,
    random_seed=1,
)

test_loader = get_test_loader(
    data_dir=data_dir,
    batch_size=64,
)


100.0%


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Files already downloaded and verified


![](image.png)

In [4]:
class Alexnet(nn.Module):
    def __init__(self,num_classes = 10): #we use CIFAR10 dataset, hence num_classes = 10 
        super(Alexnet,self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,96, kernel_size=11,stride=4,padding=0),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride=2))
        
        self.layer2 = nn.Sequential(
                nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
                nn.BatchNorm2d(256),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size = 3, stride = 2))
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        
        self.layer5 = nn.Sequential(
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(9216, 4096),
            nn.ReLU())
        
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        
        self.fc2= nn.Sequential(
            nn.Linear(4096, num_classes))
        
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        
        out = out.reshape(out.size(0), -1)
        
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [5]:
#hyperparameters 

num_classes = 10 
num_epochs = 10 
batch_size = 64
learning_rate = 0.005

model = Alexnet(num_classes).to(device)

#loss fn.
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate,weight_decay=0.005)

total_step = len(train_loader)

In [6]:
total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                    .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs

        print('Accuracy of the network on the {} validation images: {} %'.format(5000, 100 * correct / total))

Epoch [1/10], Step [704/704], Loss: 0.7973
Accuracy of the network on the 5000 validation images: 44.26 %
Epoch [2/10], Step [704/704], Loss: 0.5271
Accuracy of the network on the 5000 validation images: 43.24 %
Epoch [3/10], Step [704/704], Loss: 1.0180
Accuracy of the network on the 5000 validation images: 58.62 %
Epoch [4/10], Step [704/704], Loss: 1.4266
Accuracy of the network on the 5000 validation images: 50.88 %
Epoch [5/10], Step [704/704], Loss: 0.7761
Accuracy of the network on the 5000 validation images: 61.6 %
Epoch [6/10], Step [704/704], Loss: 0.6883
Accuracy of the network on the 5000 validation images: 60.34 %
Epoch [7/10], Step [704/704], Loss: 1.4347
Accuracy of the network on the 5000 validation images: 68.72 %
Epoch [8/10], Step [704/704], Loss: 0.5515
Accuracy of the network on the 5000 validation images: 68.32 %
Epoch [9/10], Step [704/704], Loss: 1.2966
Accuracy of the network on the 5000 validation images: 55.46 %
Epoch [10/10], Step [704/704], Loss: 0.1995
Acc